In [ ]:
import xml.etree.ElementTree as ET
import os
import tensorflow as tf

class LostAndFound:
    def __init__(self, xml_path):
      self.xml_path = xml_path
      self.tree = ET.parse(self.xml_path)
      self.found_people = {}
      self.missing_people = {}


In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=2fdc4e25ec47ef0080f9285265023b7b66fd3c020207c5ad80b7a5d443b6eb8c
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from deepface import DeepFace
import os
import uuid
from PIL import Image

25-07-10 11:30:38 - Directory /root/.deepface has been created
25-07-10 11:30:38 - Directory /root/.deepface/weights has been created


In [ ]:
def get_embedding(image_path):
   try:
      embdding_obj = DeepFace.represent(img_path= image_path,model_name='Facenet',
                                        enforce_detection=True)
      embedding = embdding_obj[0]['embedding']
      return embedding
   except ValueError as e:
      print(f"Could not process {image_path}: {e}")
      return None

In [ ]:
database_dir = 'database/'
face_database ={}

In [ ]:
def register_new_face(person_name, image):
    if not person_name or image is None:
        return "Please provide a name and image."
    person_folder = os.path.join(database_dir, person_name)
    os.makedirs(person_folder, exist_ok=True)
    filename = f"{uuid.uuid4().hex}.jpg"
    save_path = os.path.join(person_folder, filename)
    image.save(save_path)

    return f"Image saved for {person_name} at {save_path}"

In [ ]:
import gradio as gr

register_interface = gr.Interface(
    fn=register_new_face,
    inputs=[
        gr.Textbox(label="Person Name"),
        gr.Image(label="Capture Face", sources="webcam", type="pil")
    ],
    outputs=gr.Textbox(label="Status"),
    title="Register a New Face"
)

In [ ]:
def update_face_database():
    global face_database
    face_database = {}
    for person_name in os.listdir(database_dir):
      person_dir = os.path.join(database_dir,person_name)
      if os.path.isdir(person_dir):
        person_embeddings = []
        for filename in os.listdir(person_dir):
          pth = os.path.join(person_dir, filename)
          embedding = get_embedding(pth)
          if embedding is not None:
            person_embeddings.append(embedding)
        if person_embeddings:
          face_database[person_name] = np.mean(person_embeddings, axis=0)
          print(f"Added {person_name} to the database")

In [ ]:
face_database

{}

In [ ]:
def find_match(image_path, database, threshold=0.4):
  unknown_embeding = get_embedding(image_path)
  if unknown_embeding is None:
    return 'No face found in image'
  min_dist = float('inf')
  match_name ='Unknown'

  for name, db_embedding in face_database.items():
    dist = cosine(db_embedding, unknown_embeding)
    if dist < min_dist:
       min_dist = dist
       match_name = name
    if min_dist <= threshold:
      return f"Match found {match_name} (Distance : {min_dist:.4f})"
    else:
      return "No match found in database"

In [ ]:
def find_match_gradio(image):
    temp_path = "temp_input.jpg"
    image.save(temp_path)
    result = find_match(temp_path, face_database)
    return result

interface = gr.Interface(
    fn=find_match_gradio,
    inputs=gr.Image(label="Input", sources="webcam", type="pil"),
    outputs=gr.Textbox(label="Match Result"),
    live=False
)

In [ ]:
update_btn = gr.Interface(
    fn=update_face_database,
    inputs=[],
    outputs=gr.Textbox(label="Update Status"),
    title="Update Face Database"
)

dashboard = gr.TabbedInterface(
    interface_list=[register_interface, interface, update_btn],
    tab_names=["Register", "Recognize", "Update Database"]
)

In [ ]:
# dashboard.launch()

EXERCISE


In [ ]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from deepface import DeepFace
import os
import uuid
from PIL import Image

In [ ]:
def get_embedding(image_path):
  try:
    embedding_obj = DeepFace.represent(img_path = img_path,model_name='Facenet',
                                       enforce_detection=True)
    embedding = embedding_obj[0]['embedding']
    return embedding
  except valueError as e:
    print(f"Could not process {image_path}: {e}")
    return None

In [ ]:
database_dir = 'database/'
face_database = {}

In [ ]:
def register_new_face(perso_name, image):
  if not person_name or image is None:
    return "Please provide a name and image."

  person_folder = os.path.join(database_dir, prson_name)
  os.makedirs(persorn_folder, exist_ok=True)
  filename = f"{uuid.uuid4().hex}.jpg"
  save_path = os.path.join(person_folder, filename)
  image.save(save_path)

  return f"Image saved for {prson_name} at {save_path}"

In [ ]:
register_interface = gr.Interface(
    fn=register_new_face,
    inputs=[
        gr.Textbox(label="Person Name"),
        gr.Image(label="Capture Face", sources="webcam", type="pil")
    ],
    outputs=gr.Textbox(label="Status"),
    title="Register a New Face"
)

In [ ]:
def udate_face_database():
  global face_database
  face_database = {}
  for person_name in os.listdir(database_dir):
    person_dir = os.path.join(database_dir)
    if os.path.isdir(person_dir):
      person_embeddings = []
      for filename in os.listdir(person_dir):
        pth = os.path.join(person_dir, filename)
        embedding = get_embedding(pth)
        if embedding is not None:
          perrson_embeddings.append(embedding)
      if person_embeddings:
        face_databese[person_name] = np.mean(person_embeddings, axis=0)
        print(f"added {person_name} to the database")

In [ ]:
face_database

{}

In [ ]:
def find_match(image_path, database, threshold=0.4):
  unknown_embedding = get_embedding(image_path)
  if unknown_embedding is None:
    return  'No face found in image'
  min_dist = float('inf')
  match_name = 'Unknown'
  for name, db_embedding in face_database.items():
    dist = cosine(db_embedding, unknown_embedding)
    if dist < min_dist:
      min_dist = dist
      match_name = name
  if min_dist <= threshold:
    return f"Match found {match_name} (Distance : {min_dist:.4f})"
  else:
    return "No match found in database"

In [ ]:
def find_match_gradio(image):
  temp_path = "temp_input.jpg"
  image.save(temp_path)
  result = find_match(temp_path, face_database)
  return result

interface = gr.Interface(
    fn=find_match_gradio,
    inputs=gr.Image(label="Input", sources="webcam", type="pil"),
    outputs=gr.Textbox(label="Match Result"),
    live=False
)

In [ ]:
update_btn = gr.Interface(
    fn=update_face_database,
    inputs=[],
    outputs=gr.Textbox(label="Update Status"),
    title="Update Face Database"
)

dashboard = gr.TabbedInterface(
    interface_list = [register_interface, interface, update_btn],
    tab_names = ["Register", "Recognize", "Update database"]
)

In [ ]:
dashboard.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1965c992cf92418976.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
